# Create a pipeline

This notebook shows how to use the SDK to create pipelines.

We start by creating a client with our Personal Access Token.

In [2]:
from glassflow import GlassFlowClient

# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "< your-personal-access-token >"

In [3]:
client = GlassFlowClient(
    personal_access_token=personal_access_token
)

Next step is to create a transformation function to run on the pipeline. Let's create a simple transformation to print the event data in the logs.

In [4]:
%%bash
echo "Writing transformation function ..."
cat <<EOT > transformation.py 
def handler(data, log):
    log.info(data)
    return data
EOT
echo "Done!"

Writing transformation function ...
Done!


In GlassFlow, pipelines need to be created inside a space.

In [5]:
space = client.create_space(name="tutorials")
print("Space ID:", space.id)

Space ID: d9b1c8df-b95a-4ddf-9731-e27f34c9a0f7


Now, we have everything we need to create a pipeline:

In [6]:
pipeline = client.create_pipeline(
    name="example pipeline",
    space_id=space.id,
    transformation_file="transformation.py",
)
print("Pipeline ID:", pipeline.id)

Pipeline ID: bf90e3b8-a0d4-4a8b-819f-710357b5dc1e


In [7]:
from IPython.display import Markdown as md

md(f"```python\n{pipeline.transformation_code}\n```")

```python
def handler(data, log):
    log.info(data)
    return data

```

If we look at the pipeline logs, we can see that the pipeline was created successfully! 🚀

(Logs might take few seconds to come up, so you might have to run this cell few times for the logs to show up)

In [11]:
logs_res = pipeline.get_logs()
for log in logs_res.logs:
    print(log.to_json())

{"level": "INFO", "severity_code": 200, "timestamp": "2024-10-02T05:42:12.252989163Z", "payload": {"message": "Function is uploaded"}}
{"level": "INFO", "severity_code": 200, "timestamp": "2024-10-02T05:42:12.161987721Z", "payload": {"message": "Pipeline is created"}}


### Transformation Requirements

The previous example shows a basic example of pipeline. In some cases, we might need to use additional dependencies for our transformation code. We can pass the requirements.txt file to the pipeline.

In [12]:
pipeline_with_requirements = client.create_pipeline(
    name="example pipeline with requirements",
    transformation_file="transformation.py",
    space_id=space.id,
    requirements="requests==2.32.3"
)
print("Pipeline ID:", pipeline_with_requirements.id)

Pipeline ID: c439ede3-c259-446d-8ad7-0c81b7a29b5c


### Environment Variables

Pipelines support environment variables

In [13]:
pipeline_with_env_vars = client.create_pipeline(
    name="example pipeline with requirements",
    transformation_file="transformation.py",
    space_id=space.id,
    env_vars=[
        {
            "name": "MY_VAR",
            "value": "example value"
        }
    ]
)
print("Pipeline ID:", pipeline_with_env_vars.id)

Pipeline ID: 02b93a9f-e769-4114-8f95-61c8cfcd0d42


## Clean-up

Delete the pipelines and spaces we created:

In [15]:
pipeline.delete()
pipeline_with_env_vars.delete()
pipeline_with_requirements.delete()
space.delete()